In [11]:
import dataset, model
import albumentations as A
import matplotlib.pyplot as plt
import pandas as pd
import torch
from torch import nn
from torch.nn import functional as F
import numpy as np
import random, os

In [12]:
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore
    
set_seed(0) # DONT CHANGE THE SEED

In [13]:
import importlib
importlib.reload(model)
importlib.reload(dataset)

<module 'dataset' from 'D:\\f1\\ch3\\dataset.py'>

In [14]:
train = pd.read_csv('data/train.csv')
val = pd.read_csv('data/val.csv')

train.ImageID = [f'train/{i}' for i in train.ImageID]
val.ImageID = [f'val/{i}' for i in val.ImageID]
trainval = pd.concat((train,val))
trainval.to_csv('data/trainval.csv')

In [15]:
## RUN THIS CELL TWICE FOR TWO SEPERATE MODELS

import model
import albumentations as A
from albumentations.augmentations.transforms import Flip, Blur, ChannelShuffle
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

if __name__ == '__main__':
    ckpt = [ModelCheckpoint()
           ]
    trainer = Trainer(max_epochs = 100,gpus = 1, callbacks = ckpt, precision=16,deterministic=True,fast_dev_run = False)
    
    train_tr = A.Compose([
        A.augmentations.transforms.ColorJitter(),
        A.augmentations.transforms.GaussNoise(),
        #A.Resize(32,100)
        A.Affine(0.7,translate_px = 15),
        Blur(),
        A.ChannelShuffle()
    ])
    
    val_tr = A.Compose([
        #A.CenterCrop(128,128),
        #A.Resize(32,100)
    ])
    
    model = model.Classifier({'lr':3e-4,'batch_size':64,'train_tr':train_tr,'val_tr':val_tr})
    
    trainer.fit(model)
    trainer.test(model)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.

  | Name  | Type  | Params
--------------------------------
0 | model | Model | 48.9 M
--------------------------------
48.9 M    Trainable params
0         Non-trainable params
48.9 M    Total params
195.413   Total estimated model params size (MB)
Restored states from the checkpoint file at lightning_logs/version_65/checkpoints/epoch=99-step=68799.ckpt


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss': 0.0008712546550668776, 'test_mse': 9.899250030517578}
--------------------------------------------------------------------------------


In [22]:
import model
model = model.Classifier({'lr':3e-4,'batch_size':32,'train_tr':train_tr,'val_tr':val_tr})

ckpt = torch.load('lightning_logs/version_65/checkpoints/epoch=99-step=68799.ckpt') # PATH OF CHECKPOINT 1
model.load_state_dict(ckpt['state_dict'])
trainer.test(model)
out = trainer.predict(model)
# SHOULD SHOW A LOSS AROUND 0.000930169 AND A MSE OF AROUND 66.6572494. IF NOT SOMETHING WASN'T DONE CORRECTLY
# IF THE LOSS ISN'T EXACLTY 0.000930169 THE LB-SCORE WILL DIFFER SLIGHTLY

ckpt = torch.load('lightning_logs/version_66/checkpoints/epoch=97-step=67423.ckpt') # PATH OF CHECKPOINT 2
model.load_state_dict(ckpt['state_dict'])
trainer.test(model)
out2 = trainer.predict(model)
# SHOULD SHOW A LOSS AROUND 0.000918057 AND A MSE OF UNDER 50. IF NOT SOMETHING WASN'T DONE CORRECTLY
# IF THE LOSS ISN'T EXACLTY 0.000918057 THE LB-SCORE WILL DIFFER SLIGHTLY

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss': 0.0009301690733991563, 'test_mse': 66.6572494506836}
--------------------------------------------------------------------------------


Predicting: 688it [00:00, ?it/s]

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss': 0.000918057921808213, 'test_mse': 1.7582499980926514}
--------------------------------------------------------------------------------


Predicting: 688it [00:00, ?it/s]

In [17]:
def removeBatches(t):
    concat = torch.tensor([])
    for i in range(len(t)):
        concat = torch.cat((concat,torch.tensor(t[i])))
    return concat

In [18]:
import model
out = removeBatches(out)
out2 = removeBatches(out2)
out = ((out+out2)/2)
concat = model.toNum(out).squeeze()

In [19]:
import model
model.writeSub(concat) 

In [20]:
!aicrowd submission create -c f1-speed-recognition -f submission.csv

submission.csv ------------------- 100.0% • 105.1/103.5 KB • 1.5 MB/s • 0:00:00                                                   +-------------------------+                                                    
                                                   | Successfully submitted! |                                                    
                                                   +-------------------------+                                                    
                                                         Important links                                                          
+--------------------------------------------------------------------------------------------------------------------------------+
|  This submission | https://www.aicrowd.com/challenges/ai-blitz-8/problems/f1-speed-recognition/submissions/140346              |
|                  |                                                                                                             |
|  A